In [1]:
# Load all required library
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import re
from bs4 import BeautifulSoup
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from gensim.models import Word2Vec
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import torch
from torch.utils.data import DataLoader, Dataset, TensorDataset
import torch.nn as nn
import torch.nn.functional as F

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
# from google.colab import drive
# drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# 1. Dataset Generation

In [3]:
path = './data.tsv'
# path = '/content/drive/MyDrive/CSCI544/amazon_reviews_us_Office_Products_v1_00.tsv'
# read raw data set
raw_data = pd.read_table(path, on_bad_lines='skip')

<ipython-input-3-5b6687d1ed64>:4: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  raw_data = pd.read_table(path, on_bad_lines='skip')


In [4]:
# data cleaning and preprocessing
new_data = raw_data[['star_rating', 'review_body']].copy()

# Lowercase
new_data['review_body'] = new_data['review_body'].str.lower()

# remove the HTML and URLs from the reviews
def remove_html(review):
    if isinstance(review, str):
        soup = BeautifulSoup(review, 'html.parser')
        text = soup.get_text()
        return re.sub(r'https?://\S+|www\.\S+', '', text)
    else:
        return review

new_data['review_body'] = new_data['review_body'].apply(remove_html)

# remove non-alphabetical characters
new_data['review_body'] = new_data['review_body'].str.replace(r'[^a-z\s]', '',regex=True)

# Remove numbers and floats
def remove_float(string):
    return re.sub(r'[-+]?\d*\.\d+|\d+', '', str(string))
new_data['review_body'] = new_data['review_body'].apply(remove_float)

# remove extra spaces
def remove_spaces(string):
    return ' '.join(string.split())

new_data['review_body'] = new_data['review_body'].apply(remove_spaces)

# remove null values
new_data = new_data.dropna()

<ipython-input-4-db5226ba4bb8>:10: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(review, 'html.parser')
<ipython-input-4-db5226ba4bb8>:10: MarkupResemblesLocatorWarning: The input looks more like a URL than markup. You may want to use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  soup = BeautifulSoup(review, 'html.parser')


In [5]:
new_data['class'] = 0
# Let ratings with the values of 1, 2 and 3 form class 1
new_data.loc[new_data['star_rating'].isin([1, 2, 3]), 'class'] = 1
# Let ratings with the values of 4 and 5 form class 2
new_data.loc[new_data['star_rating'].isin([4, 5]), 'class'] = 2
# Delete entries with invalid ratings
new_data = new_data[new_data['class'] != 0]

# randomly select 50000 reviews from each class to build a balanced dataset of 100K reviews
class1 = new_data[new_data['class'] == 1].sample(n = 50000, random_state=7, ignore_index=True)
class2 = new_data[new_data['class'] == 2].sample(n = 50000, random_state=7, ignore_index=True)
samples = pd.concat([class1, class2], axis=0).reset_index(drop=True)

# # split data into training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(samples['review_body'], samples['class'],
                                                    test_size=0.2, random_state=7)


# 2. Word Embedding

In [6]:
# Load the pretrained “word2vec-google-news-300” Word2Vec model
import gensim.downloader as api
wv = api.load('word2vec-google-news-300')


(a) check semantic similarities

first example: sky - cloud + sun =

In [7]:
similar_word = wv.most_similar(positive=['sky', 'sun'], negative=['cloud'], topn=1)
print(f"Words similar to 'sky - cloud + sun': {similar_word}")

Words similar to 'sky - cloud + sun': [('sunlight', 0.528194010257721)]


second example: children~kids

In [8]:
similarity_score = wv.similarity('children', 'kids')
print("Similarity score between 'children' and 'kids':", similarity_score)

Similarity score between 'children' and 'kids': 0.743905


thrid example: beautiful~gorgeous

In [9]:
similarity_score = wv.similarity('beautiful', 'gorgeous')
print("Similarity score between 'beautiful' and 'gorgeous':", similarity_score)

Similarity score between 'beautiful' and 'gorgeous': 0.8353004


(b) Train a Word2Vec model using your own dataset

In [10]:
# tokenize data
tokenized_review = samples['review_body'].apply(word_tokenize)

my_w2v = Word2Vec(tokenized_review, vector_size=300, window=13, min_count=9)

Check the semantic similarities for the same two examples in part (a)

first example: sky - cloud + sun =

In [11]:
similar_word = my_w2v.wv.most_similar(positive=['sky', 'sun'], negative=['cloud'], topn=1)
print(f"Words similar to 'sky - cloud + sun': {similar_word}")

Words similar to 'sky - cloud + sun': [('glow', 0.6744972467422485)]


second example: beautiful~gorgeous

In [12]:
similarity_score = my_w2v.wv.similarity('beautiful', 'gorgeous')
print("Similarity score between 'beautiful' and 'gorgeous':", similarity_score)

Similarity score between 'beautiful' and 'gorgeous': 0.7948074


What do you conclude from comparing vectors generated by yourself and the pretrained model?


> Both your custom model and the pretrained model show a high similarity score between 'beautiful' and 'gorgeous', while my model gives a similar score of 0.818 and the pretrained model gives 0.835. This suggests that both models have learned a similar relationship between these two words, indicating that they are semantically similar in the context they were trained on. For words similar to 'sun - cloud + sun', my model outputs '[('glow', 0.6235448122024536)]' and the pretrained model outputs '[('sunlight', 0.528194010257721)].'  In both cases, the results make some sense in the context of the operation, even though they are not exactly the same. Overall, the results show that both models are capable of capturing semantic relationships between words. However, the pretrained model performs slightly better because of its vast and diverse scale of training data.

Which of the Word2Vec models seems to encode semantic similarities between words better?


> The Google pre-trained Word2Vec model seems to encode semantic similarities between words better. In the example of 'sky - cloud + sun', the Google pre-trained Word2Vec generates 'sunlight', while my model generates 'glow'. In the given context, 'sunlight' will be an answer.



#3. Simple models

generate vectors


In [13]:
# calculate the average Word2Vec vectors for each review
def getVectors(sentence):
    tokenized = word_tokenize(sentence)
    num_words = 0
    total = np.zeros(300)
    for word in tokenized:
        if word in wv:
            total += wv[word]
            num_words += 1

    if num_words > 0:
        average_vector = total / num_words
    else:
        average_vector = np.zeros(300)
    return average_vector


X_train_w2v = np.array([getVectors(review) for review in X_train])
X_test_w2v = np.array([getVectors(review) for review in X_test])

In [14]:
# TF-IDF Feature Extraction
tfidf_vectorizer = TfidfVectorizer()
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

perceptron model

In [15]:
from sklearn.linear_model import Perceptron
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

perceptron = Perceptron(random_state=6)

# train a perceptron model using word2vec feature
perceptron.fit(X_train_w2v, y_train)
preceptron_predict = perceptron.predict(X_test_w2v)

# Report Precision, Recall, and f1-score for word2vec features.
w2v_accuracy = accuracy_score(y_test, preceptron_predict)
# w2v_precision = precision_score(y_test, preceptron_predict)
# w2v_recall = recall_score(y_test, preceptron_predict)
# w2v_f1 = f1_score(y_test, preceptron_predict)

print('Report for training Perceptron model using word2vec features:')
# print(f'Precision: {w2v_precision:.3f}; Recall: {w2v_recall:.3f}; F1: {w2v_f1:.3f}')
print(f'Accuracy: {w2v_accuracy:.3f}.')

# train a perceptron model for tfidf
perceptron.fit(X_train_tfidf, y_train)
preceptron_predict = perceptron.predict(X_test_tfidf)

# Report Precision, Recall, and f1-score
tfidf_accuracy = accuracy_score(y_test, preceptron_predict)
# tfidf_precision = precision_score(y_test, preceptron_predict)
# tfidf_recall = recall_score(y_test, preceptron_predict)
# tfidf_f1 = f1_score(y_test, preceptron_predict)

print('Report for training Perceptron model using TFIDF features:')
# print(f'Precision: {tfidf_precision:.3f}; Recall: {tfidf_recall:.3f}; F1: {tfidf_f1:.3f}')
print(f'Accuracy: {tfidf_accuracy:.3f}.')

Report for training Perceptron model using word2vec features:
Accuracy: 0.799.
Report for training Perceptron model using TFIDF features:
Accuracy: 0.808.


SVM model

In [16]:
from sklearn.svm import LinearSVC

svm_model = LinearSVC(random_state=6, max_iter=100000, tol=1e-6, dual=True)
# train svm model using word2vec features
svm_model.fit(X_train_w2v, y_train)
svm_pred = svm_model.predict(X_test_w2v)

# Report Precision, Recall, and f1-score for training SVM using word2vec features.
w2v_accuracy = accuracy_score(y_test, svm_pred)
# w2v_precision = precision_score(y_test, svm_pred)
# w2v_recall = recall_score(y_test, svm_pred)
# w2v_f1 = f1_score(y_test, svm_pred)

print('Report for training SVM model using word2vec features:')
# print(f'Precision: {w2v_precision:.3f}; Recall: {w2v_recall:.3f}; F1: {w2v_f1:.3f}')
print(f'Accuracy: {w2v_accuracy:.3f}.')

# train svm model using tfidf features
svm_model.fit(X_train_tfidf, y_train)
svm_pred = svm_model.predict(X_test_tfidf)

# Report Precision, Recall, and f1-score for training SVM using tfidf features.
tfidf_accuracy = accuracy_score(y_test, svm_pred)
tfidf_precision = precision_score(y_test, svm_pred)
tfidf_recall = recall_score(y_test, svm_pred)
tfidf_f1 = f1_score(y_test, svm_pred)

print('Report for training SVM model using TFIDF features:')
# print(f'Precision: {tfidf_precision:.3f}; Recall: {tfidf_recall:.3f}; F1: {tfidf_f1:.3f}')
print(f'Accuracy: {tfidf_accuracy:.3f}.')

Report for training SVM model using word2vec features:
Accuracy: 0.816.
Report for training SVM model using TFIDF features:
Accuracy: 0.866.


What do you conclude from comparing performances for the models trained using the two different feature types?


>The accuracy values for both model have shown that the models trained using TFIDF features have a slightly better performance.



#4. Feedforward Neural Networks

(a)

prepare data

In [17]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define custom dataset class
class dataset(Dataset):
  def __init__(self,x,y):
    self.x = torch.tensor(x,dtype=torch.float32)
    self.y = torch.tensor(y,dtype=torch.int64) - 1

  def __getitem__(self,idx):
    return self.x[idx],self.y[idx]

  def __len__(self):
    return len(self.x)


In [18]:
y_test = y_test.to_numpy()

trainset = dataset(X_train_w2v, y_train)
testset = dataset(X_test_w2v, y_test)

#DataLoader
trainloader = DataLoader(trainset, batch_size=32, shuffle=True)
testloader = DataLoader(testset, batch_size=1, shuffle=False)

create the model

In [19]:
# Define a feedforward neural network architechture
class Net(nn.Module):
    def __init__(self, input_size):
        super(Net, self).__init__()
        # number of hidden nodes in each layer
        hidden_1 = 50
        hidden_2 = 5
        self.fc1 = nn.Linear(input_size, hidden_1)
        self.fc2 = nn.Linear(hidden_1, hidden_2)
        # output layer
        self.output = nn.Linear(hidden_2, 2)
        # define drop out layer
        self.dropout = nn.Dropout(0.2)

    def forward(self, x):
        # add hidden layer, with relu activation function
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.dropout(x)
        x = self.output(x)
        return x


train the network

In [20]:
# initiate model
FNNa = Net(300).to(device)

# specify loss function (categorical cross-entropy)
criterion = nn.CrossEntropyLoss()

# specify optimizer
learning_rate = 0.001
# optimizer = torch.optim.SGD(FNNa.parameters(), lr=learning_rate)
optimizer = torch.optim.Adam(FNNa.parameters(), lr=learning_rate)

# training the network
# number of epochs to train the model
n_epochs = 20

for epoch in range(n_epochs):
    FNNa.train()
    for data, target in trainloader:
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = FNNa(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()

    # Print loss for every epoch
    print(f"Epoch [{epoch + 1}/{n_epochs}], Loss: {loss.item():.4f}")


Epoch [1/20], Loss: 0.6795
Epoch [2/20], Loss: 0.5770
Epoch [3/20], Loss: 0.4137
Epoch [4/20], Loss: 0.3509
Epoch [5/20], Loss: 0.6418
Epoch [6/20], Loss: 0.3148
Epoch [7/20], Loss: 0.2354
Epoch [8/20], Loss: 0.3913
Epoch [9/20], Loss: 0.3925
Epoch [10/20], Loss: 0.3883
Epoch [11/20], Loss: 0.2705
Epoch [12/20], Loss: 0.3290
Epoch [13/20], Loss: 0.4074
Epoch [14/20], Loss: 0.4191
Epoch [15/20], Loss: 0.2520
Epoch [16/20], Loss: 0.2000
Epoch [17/20], Loss: 0.3561
Epoch [18/20], Loss: 0.2708
Epoch [19/20], Loss: 0.4137
Epoch [20/20], Loss: 0.2196


make prediction on test set and report accuracy score

In [21]:
def eval(model, dataloader):
    prediction_list = []
    true_list = []
    for i, (x, y) in enumerate(dataloader):
        x = x.to(device)
        outputs = model(x)
        _, predicted = torch.max(outputs, 1)
        prediction_list.extend(predicted.cpu().numpy())
        true_list.extend(y.numpy())
    return accuracy_score(true_list, prediction_list)

w2v_accuracy = eval(FNNa,testloader)
print('Report for training Feedforward Neural Network using the average Word2Vec vectors:')
print(f'Accuracy: {w2v_accuracy:.3f}.')

Report for training Feedforward Neural Network using the average Word2Vec vectors:
Accuracy: 0.839.


(b)

generate input vectors

In [22]:
# calculate the first 10 Word2Vec vectors for each review
def getTenVectors(sentence):
    tokenized = word_tokenize(sentence)
    total = np.array([])
    count = 0
    for word in tokenized:
        if count < 10:
            if word in wv:
                total = np.concatenate((total, wv[word]))
            else:
                total = np.concatenate((total, np.zeros(300)))
            count += 1
        else:
            break

    while len(total) < 3000:
         total = np.concatenate((total, np.zeros(300)))
    return total

X_train_10_w2v = np.array([getTenVectors(review) for review in X_train])
X_test_10_w2v = np.array([getTenVectors(review) for review in X_test])

prepare the data

In [23]:
trainset = dataset(X_train_10_w2v, y_train)
testset = dataset(X_test_10_w2v, y_test)

#DataLoader
trainloader = DataLoader(trainset, batch_size=32, shuffle=True)
testloader = DataLoader(testset, batch_size=1, shuffle=False)

train the network

We can use the same Net class defined in part (a) to build a feedforward neural network, but for part (b), the input size is changed to 3000.

In [24]:
# initiate the network
FNNb = Net(3000).to(device)

# specify loss function (categorical cross-entropy)
criterion = nn.CrossEntropyLoss()

# specify optimizer
learning_rate = 0.001
# optimizer = torch.optim.SGD(FNNa.parameters(), lr=learning_rate)
optimizer = torch.optim.Adam(FNNb.parameters(), lr=learning_rate)

# training the network
# number of epochs to train the model
n_epochs = 20

for epoch in range(n_epochs):
    FNNb.train()
    for data, target in trainloader:
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = FNNb(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()

    # Print loss for every epoch
    print(f"Epoch [{epoch + 1}/{n_epochs}], Loss: {loss.item():.4f}")


Epoch [1/20], Loss: 0.5302
Epoch [2/20], Loss: 0.4463
Epoch [3/20], Loss: 0.5834
Epoch [4/20], Loss: 0.5409
Epoch [5/20], Loss: 0.4429
Epoch [6/20], Loss: 0.2793
Epoch [7/20], Loss: 0.2628
Epoch [8/20], Loss: 0.2073
Epoch [9/20], Loss: 0.1963
Epoch [10/20], Loss: 0.3585
Epoch [11/20], Loss: 0.1140
Epoch [12/20], Loss: 0.0749
Epoch [13/20], Loss: 0.0960
Epoch [14/20], Loss: 0.1955
Epoch [15/20], Loss: 0.0642
Epoch [16/20], Loss: 0.1241
Epoch [17/20], Loss: 0.1422
Epoch [18/20], Loss: 0.0950
Epoch [19/20], Loss: 0.0892
Epoch [20/20], Loss: 0.1255


make prediction on test set and report accuracy score

In [25]:
w2v_accuracy = eval(FNNb,testloader)
print('Report for training Feedforward Neural Network using the first 10 Word2Vec vectors:')
print(f'Accuracy: {w2v_accuracy:.3f}.')

Report for training Feedforward Neural Network using the first 10 Word2Vec vectors:
Accuracy: 0.731.


What do you conclude by comparing accuracy values you obtain with
those obtained in the “Simple Models” section.


> The feedforward neural network trained using the average Word2Vec vectors has a better accuracy value compared to both models in the “Simple Models” section. Thus, with the same training data, feedforward neural network has a better performance over the “Simple Models”. However, the feedforward neural network trained using the first 10 Word2Vec vectors perform worse than both models in the “Simple Models” section.



#5. Recurrent Neural Networks

(a)

prepare data

For preparing the data, this part will be using the custom dataset class defined in part 4.

In [26]:
def truncate(sentence):
    tokenized = word_tokenize(sentence)
    total = np.zeros((10,300))
    for i, word in enumerate(tokenized):
        if i >= 10:
            break
        if word in wv:
          total[i] = wv[word]

    return total

X_train_rnn = np.array([truncate(review) for review in X_train])
X_test_rnn = np.array([truncate(review) for review in X_test])

In [27]:
trainset = dataset(X_train_rnn, y_train)
testset = dataset(X_test_rnn, y_test)

#DataLoader
trainloader = DataLoader(trainset, batch_size=32, shuffle=True)
testloader = DataLoader(testset, batch_size=1, shuffle=False)


define model architecture

In [28]:
class SimpleRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(SimpleRNN, self).__init__()
        self.hidden_size = hidden_size
        # rnn layer
        self.rnn = nn.RNN(input_size, hidden_size, nonlinearity='relu', dropout=0.2, batch_first=True)
        # output layer
        self.output = nn.Linear(hidden_size, 2)

    def forward(self, x):
        # Initialize hidden state with zeros
        h0 = torch.zeros(1, x.size(0), 10).to(device)
        out, _ = self.rnn(x, h0)
        out = self.output(out[:, -1, :])
        return out


train the network

In [29]:
hidden_size = 10
RNN = SimpleRNN(300, hidden_size).to(device)
criterion = nn.CrossEntropyLoss()
learning_rate = 0.001
optimizer = torch.optim.Adam(RNN.parameters(), lr=learning_rate)

# Train the network
num_epochs = 20
for epoch in range(num_epochs):
    for i, (x, y) in enumerate(trainloader):
        x, y = x.to(device), y.to(device)

        outputs = RNN(x)
        loss = criterion(outputs, y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/rnn.py:71: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


Epoch [1/20], Loss: 0.6175
Epoch [2/20], Loss: 0.4481
Epoch [3/20], Loss: 0.4493
Epoch [4/20], Loss: 0.4393
Epoch [5/20], Loss: 0.4596
Epoch [6/20], Loss: 0.4626
Epoch [7/20], Loss: 0.3558
Epoch [8/20], Loss: 0.5406
Epoch [9/20], Loss: 0.4424
Epoch [10/20], Loss: 0.4309
Epoch [11/20], Loss: 0.3818
Epoch [12/20], Loss: 0.3864
Epoch [13/20], Loss: 0.5633
Epoch [14/20], Loss: 0.4773
Epoch [15/20], Loss: 0.4252
Epoch [16/20], Loss: 0.3958
Epoch [17/20], Loss: 0.5686
Epoch [18/20], Loss: 0.6625
Epoch [19/20], Loss: 0.4233
Epoch [20/20], Loss: 0.3666


make prediction on test set and report accuracy score

For model evaluation and reporting accuracy value, this part will be using the same eval function defined in part 4 to generate the accuracy score for each model.

In [30]:
rnn_accuracy = eval(RNN,testloader)

print('Report for training Recurrent Neural Network using the average Word2Vec vectors:')
print(f'Accuracy: {rnn_accuracy:.3f}.')

Report for training Recurrent Neural Network using the average Word2Vec vectors:
Accuracy: 0.780.


What do you conclude by comparing accuracy values you obtain with those obtained with feedforward neural network models.


> After comparing the accuracy values, it is shown that for models trained using the same average word2vec vectors, the Feedforward Neural Network has a better performance. The RNN network performs better than the FNN that is trained on the first 10 word vectors.



(b)

model architecture

In [31]:
class GRUNetwork(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(GRUNetwork, self).__init__()
        self.hidden_size = hidden_size
        self.gru = nn.GRU(input_size, hidden_size, batch_first=True, dropout=0.1)
        self.output = nn.Linear(hidden_size, 2)
        self.relu = nn.ReLU()

    def forward(self, x):
        h0 = torch.zeros(1, x.size(0), self.hidden_size).to(device)
        out, _ = self.gru(x, h0)
        out = self.output(self.relu(out[:, -1, :]))
        return out


train the network

In [32]:
hidden_size = 10
GRU = GRUNetwork(300, hidden_size).to(device)
criterion = nn.CrossEntropyLoss()
learning_rate = 0.001
optimizer = torch.optim.Adam(GRU.parameters(), lr=learning_rate)

# Train the network
num_epochs = 30
for epoch in range(num_epochs):
    for i, (x, y) in enumerate(trainloader):
        x, y = x.to(device), y.to(device)
        outputs = GRU(x)
        loss = criterion(outputs, y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/rnn.py:71: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


Epoch [1/30], Loss: 0.5030
Epoch [2/30], Loss: 0.5049
Epoch [3/30], Loss: 0.4333
Epoch [4/30], Loss: 0.4125
Epoch [5/30], Loss: 0.5265
Epoch [6/30], Loss: 0.8146
Epoch [7/30], Loss: 0.5095
Epoch [8/30], Loss: 0.3355
Epoch [9/30], Loss: 0.3028
Epoch [10/30], Loss: 0.3275
Epoch [11/30], Loss: 0.3502
Epoch [12/30], Loss: 0.4136
Epoch [13/30], Loss: 0.4535
Epoch [14/30], Loss: 0.3637
Epoch [15/30], Loss: 0.5132
Epoch [16/30], Loss: 0.4883
Epoch [17/30], Loss: 0.2734
Epoch [18/30], Loss: 0.4661
Epoch [19/30], Loss: 0.3088
Epoch [20/30], Loss: 0.5329
Epoch [21/30], Loss: 0.1946
Epoch [22/30], Loss: 0.3369
Epoch [23/30], Loss: 0.4002
Epoch [24/30], Loss: 0.3482
Epoch [25/30], Loss: 0.5557
Epoch [26/30], Loss: 0.2849
Epoch [27/30], Loss: 0.3148
Epoch [28/30], Loss: 0.3150
Epoch [29/30], Loss: 0.3127
Epoch [30/30], Loss: 0.2642


make prediction on test set and report accuracy score

In [33]:
gru_accuracy = eval(GRU,testloader)

print('Report for training Recurrent Neural Network with gated recurrent unit cell using the average Word2Vec vectors:')

print(f'Accuracy: {gru_accuracy:.3f}.')

Report for training Recurrent Neural Network with gated recurrent unit cell using the average Word2Vec vectors:
Accuracy: 0.790.


(c)

model architecture

In [38]:
class LSTMNetwork(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(LSTMNetwork, self).__init__()
        self.hidden_size = hidden_size
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True, dropout=0.2)
        self.output = nn.Linear(hidden_size, 2)
        self.relu = nn.ReLU()

    def forward(self, x):
        h0 = torch.zeros(1, x.size(0), self.hidden_size).to(device)
        c0 = torch.zeros(1, x.size(0), self.hidden_size).to(x.device)
        out, _ = self.lstm(x, (h0,c0))
        out = self.output(self.relu(out[:, -1, :]))
        return out


train the network

In [39]:
hidden_size = 10
LSTM = LSTMNetwork(300, hidden_size).to(device)
criterion = nn.CrossEntropyLoss()
learning_rate = 0.001
optimizer = torch.optim.Adam(LSTM.parameters(), lr=learning_rate)

# Train the network
num_epochs = 20
for epoch in range(num_epochs):
    for i, (x, y) in enumerate(trainloader):
        x, y = x.to(device), y.to(device)
        outputs = LSTM(x)
        loss = criterion(outputs, y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/rnn.py:71: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


Epoch [1/20], Loss: 0.4985
Epoch [2/20], Loss: 0.3211
Epoch [3/20], Loss: 0.4579
Epoch [4/20], Loss: 0.3009
Epoch [5/20], Loss: 0.4059
Epoch [6/20], Loss: 0.4050
Epoch [7/20], Loss: 0.3370
Epoch [8/20], Loss: 0.4888
Epoch [9/20], Loss: 0.4017
Epoch [10/20], Loss: 0.3831
Epoch [11/20], Loss: 0.3271
Epoch [12/20], Loss: 0.5342
Epoch [13/20], Loss: 0.1899
Epoch [14/20], Loss: 0.4108
Epoch [15/20], Loss: 0.3435
Epoch [16/20], Loss: 0.5865
Epoch [17/20], Loss: 0.2926
Epoch [18/20], Loss: 0.3920
Epoch [19/20], Loss: 0.2663
Epoch [20/20], Loss: 0.3652


make prediction on test set and report accuracy score

In [40]:
lstm_accuracy = eval(LSTM, testloader)

print('Report for training Recurrent Neural Network with LSTM unit cell using the average Word2Vec vectors:')

print(f'Accuracy: {lstm_accuracy:.3f}.')

Report for training Recurrent Neural Network with LSTM unit cell using the average Word2Vec vectors:
Accuracy: 0.796.


What do you conclude by comparing accuracy values you obtain by GRU,
LSTM, and simple RNN.


> The accuracy scores reveal that LSTM outperforms the other two models, with GRU coming in second, and RNN trailing as the least accurate. It's worth noting that their accuracy scores are quite similar, all hovering around the 0.79 mark. As a result, I would like to emphasize that the performance of these three models is relatively consistent, with LSTM showing a slight advantage.

